In [2]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

In [3]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/model'

In [4]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
word_state_size = 100
other_state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 10

In [5]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

In [6]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

In [7]:
with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

In [8]:
with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [9]:
word_hidden_state = tf.zeros([batch_size, word_state_size], name='word_hidden_state')
word_cell_state = tf.zeros([batch_size, word_state_size], name='word_cell_state')
word_init_state = tf.contrib.rnn.LSTMStateTuple(word_hidden_state, word_cell_state)

In [10]:
other_hidden_states = tf.zeros([channels-1, batch_size, other_state_size], name="hidden_state")
other_cell_states = tf.zeros([channels-1, batch_size, other_state_size], name="cell_state")

In [11]:
other_init_states = [tf.contrib.rnn.LSTMStateTuple(other_hidden_states[i], other_cell_states[i]) for i in range(channels-1)]

In [12]:
with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=word_init_state)
    state_series_word1 = tf.reduce_max(state_series, axis=1)

In [13]:
with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=word_init_state)
    state_series_word2 = tf.reduce_max(state_series, axis=1)

In [14]:
with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=other_init_states[0])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

In [15]:
with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=other_init_states[0])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

In [16]:
with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=other_init_states[1])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

In [17]:
with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=other_init_states[1])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

In [18]:
state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

In [19]:
state_series = tf.concat([state_series1, state_series2], 1)

In [20]:
state_series

<tf.Tensor 'concat_2:0' shape=(10, 600) dtype=float32>

In [21]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

In [22]:
with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [23]:
global_step = tf.Variable(0, name="global_step")

In [24]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)


In [25]:
with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

In [26]:
with tf.name_scope("accuracy"):
    correct_predictions = tf.equal(predictions, tf.arg_max(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy") 

In [27]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [28]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

In [29]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

In [30]:
word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

In [31]:
unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

In [32]:
f = open('data/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

In [33]:
relations = []
for line in open('data/train_relation.txt'):
    relations.append(line.strip().split()[1])

In [34]:
num_batches = int(8000/batch_size)
num_batches

800

In [35]:
pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

In [36]:
dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

In [37]:
relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

In [38]:
rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

In [39]:
pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

In [40]:
dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

In [41]:
pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

In [42]:
dep2id['OTH'] = 20
id2dep[20] = 'OTH'

In [43]:
JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

In [44]:
 def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [45]:
dep = []
for i in range(8000):
    dep.append(dep_p1[i])

In [46]:
for i in range(8000):
    dep.append(dep_p2[i])

In [47]:
dep_freq = nltk.FreqDist(chain(*dep))
dep_types = dep_freq.most_common(20)

In [48]:
for i in range(8000):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

In [49]:
word_p1_ids = np.ones([8000, max_len_path],dtype=int)
word_p2_ids = np.ones([8000, max_len_path],dtype=int)
pos_p1_ids = np.ones([8000, max_len_path],dtype=int)
pos_p2_ids = np.ones([8000, max_len_path],dtype=int)
dep_p1_ids = np.ones([8000, max_len_path],dtype=int)
dep_p2_ids = np.ones([8000, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

In [51]:
for i in range(8000):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [52]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [53]:
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [54]:
model = tf.train.latest_checkpoint(model_dir)
saver.restore(sess, model)

INFO:tensorflow:Restoring parameters from checkpoint/model/model-8000


In [55]:
# latest_embd = tf.train.latest_checkpoint(word_embd_dir)
# word_embedding_saver.restore(sess, latest_embd)

In [54]:
num_epochs = 10
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        if step % 10 ==0:
            print("Step:", step, "loss:",loss)
        if step % 100 == 0:
            saver.save(sess, model_dir + '/model', global_step = step)
            print("Saved Model")

Step: 10 loss: 3.07525
Step: 20 loss: 2.91253
Step: 30 loss: 2.77473
Step: 40 loss: 2.72092
Step: 50 loss: 2.94963
Step: 60 loss: 2.64186
Step: 70 loss: 2.49993
Step: 80 loss: 2.45959
Step: 90 loss: 2.52036
Step: 100 loss: 2.30564
Saved Model
Step: 110 loss: 2.24492
Step: 120 loss: 2.48354
Step: 130 loss: 1.99022
Step: 140 loss: 1.59583
Step: 150 loss: 2.18596
Step: 160 loss: 2.39128
Step: 170 loss: 1.92281
Step: 180 loss: 2.26467
Step: 190 loss: 2.24388
Step: 200 loss: 2.50149
Saved Model
Step: 210 loss: 1.8673
Step: 220 loss: 1.57685
Step: 230 loss: 2.0235
Step: 240 loss: 1.47464
Step: 250 loss: 1.51098
Step: 260 loss: 1.91565
Step: 270 loss: 1.46834
Step: 280 loss: 1.24425
Step: 290 loss: 1.74657
Step: 300 loss: 1.32431
Saved Model
Step: 310 loss: 1.72367
Step: 320 loss: 1.5951
Step: 330 loss: 1.41446
Step: 340 loss: 2.0474
Step: 350 loss: 2.63629
Step: 360 loss: 1.9582
Step: 370 loss: 1.17965
Step: 380 loss: 1.48109
Step: 390 loss: 1.8047
Step: 400 loss: 1.37329
Saved Model
Step: 4

Step: 3120 loss: 0.12847
Step: 3130 loss: 0.596706
Step: 3140 loss: 0.634581
Step: 3150 loss: 0.164035
Step: 3160 loss: 0.103016
Step: 3170 loss: 0.226977
Step: 3180 loss: 0.31817
Step: 3190 loss: 0.117716
Step: 3200 loss: 0.445029
Saved Model
Step: 3210 loss: 0.178751
Step: 3220 loss: 0.290206
Step: 3230 loss: 0.42809
Step: 3240 loss: 0.626906
Step: 3250 loss: 0.180218
Step: 3260 loss: 0.107408
Step: 3270 loss: 0.314676
Step: 3280 loss: 0.319207
Step: 3290 loss: 0.203178
Step: 3300 loss: 0.171296
Saved Model
Step: 3310 loss: 0.195655
Step: 3320 loss: 0.42697
Step: 3330 loss: 0.161593
Step: 3340 loss: 0.0945418
Step: 3350 loss: 0.14084
Step: 3360 loss: 0.757847
Step: 3370 loss: 0.137735
Step: 3380 loss: 0.278729
Step: 3390 loss: 0.152857
Step: 3400 loss: 0.127166
Saved Model
Step: 3410 loss: 0.0983252
Step: 3420 loss: 0.102324
Step: 3430 loss: 0.190508
Step: 3440 loss: 0.20741
Step: 3450 loss: 0.196798
Step: 3460 loss: 0.113721
Step: 3470 loss: 0.12239
Step: 3480 loss: 0.208887
Step: 3

Step: 6100 loss: 0.0729333
Saved Model
Step: 6110 loss: 0.077366
Step: 6120 loss: 0.0713063
Step: 6130 loss: 0.0759162
Step: 6140 loss: 0.0737381
Step: 6150 loss: 0.0857024
Step: 6160 loss: 0.0750679
Step: 6170 loss: 0.0714396
Step: 6180 loss: 0.0829965
Step: 6190 loss: 0.0785289
Step: 6200 loss: 0.0742677
Saved Model
Step: 6210 loss: 0.0713263
Step: 6220 loss: 0.0718791
Step: 6230 loss: 0.0727806
Step: 6240 loss: 0.0765687
Step: 6250 loss: 0.141632
Step: 6260 loss: 0.0692255
Step: 6270 loss: 0.0718845
Step: 6280 loss: 0.0860427
Step: 6290 loss: 0.0756211
Step: 6300 loss: 0.0696329
Saved Model
Step: 6310 loss: 0.0779379
Step: 6320 loss: 0.069084
Step: 6330 loss: 0.215432
Step: 6340 loss: 0.0814476
Step: 6350 loss: 0.0761775
Step: 6360 loss: 0.0702846
Step: 6370 loss: 0.0804309
Step: 6380 loss: 0.0787294
Step: 6390 loss: 0.0685218
Step: 6400 loss: 0.0822539
Saved Model
Step: 6410 loss: 0.0717715
Step: 6420 loss: 0.0795451
Step: 6430 loss: 0.0729375
Step: 6440 loss: 0.0690055
Step: 6450 

In [56]:
# test 
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

accuracy 99.475


In [57]:
y_pred = []
for i in range(800):
    for pred in all_predictions[i]:
        y_pred.append(pred)

In [59]:
count = 0
for i in range(8000):
    count += y_pred[i]==rel_ids[i]
accuracy = count/80

In [62]:
print(accuracy)

99.475


In [66]:
F1 = f1_score(rel_ids, y_pred, average='micro')*100

In [67]:
F1

99.475000000000009